In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.decomposition import KernelPCA as skKernelPCA

### Implementation 1
- linear kernel
- similar to sklearn kernel='linear'

In [2]:
class KernelCenterer():
    def fit(self, K):
        n_samples = K.shape[0]
        self.K_fit_rows_ = np.sum(K, axis=0) / n_samples
        self.K_fit_all_ = self.K_fit_rows_.sum() / n_samples
        return self

    def transform(self, K):
        Kt = (K - (np.sum(K, axis=1) / K.shape[1])[:, np.newaxis]
              - self.K_fit_rows_ + self.K_fit_all_)
        return Kt

In [3]:
class KernelPCA():
    def __init__(self, n_components):
        self.n_components = n_components

    @staticmethod
    def _linear_kernel(X, Y):
        K = np.dot(X, Y.T)
        return K

    def fit(self, X):
        self.X_fit_ = X
        K = self._linear_kernel(X, X)
        self._centerer = KernelCenterer().fit(K)
        Kt = self._centerer.transform(K)
        eigval, eigvec = np.linalg.eigh(Kt)
        self.lambdas_ = eigval[-self.n_components:][::-1]
        self.alphas_ = eigvec[:, -self.n_components:][:, ::-1]
        return self

    def transform(self, X):
        K = self._linear_kernel(X, self.X_fit_)
        Kt = self._centerer.transform(K)
        scaled_alphas = self.alphas_ / np.sqrt(self.lambdas_)
        return np.dot(Kt, scaled_alphas)

In [4]:
X, _ = load_iris(return_X_y=True)
kpca1 = KernelPCA(n_components=2).fit(X)
kpca2 = skKernelPCA(n_components=2).fit(X)
assert np.allclose(kpca1.lambdas_, kpca2.lambdas_)
for i in range(kpca1.alphas_.shape[1]):
    assert np.allclose(kpca1.alphas_[:, i], kpca2.alphas_[:, i]) or np.allclose(kpca1.alphas_[:, i], -kpca2.alphas_[:, i])
Xt1 = kpca1.transform(X)
Xt2 = kpca2.transform(X)
for i in range(Xt1.shape[1]):
    assert np.allclose(Xt1[:, i], Xt2[:, i]) or np.allclose(Xt1[:, i], -Xt2[:, i])

### Implementation 2
- rbf kernel
- similar to sklearn kernel='rbf'

In [5]:
class KernelPCA():
    def __init__(self, n_components, gamma=None):
        self.n_components = n_components
        self.gamma = gamma

    @staticmethod
    def _rbf_kernel(X, Y, gamma):
        if gamma is None:
            gamma = 1 / X.shape[1]
        K = np.zeros((X.shape[0], Y.shape[0]))
        for i in range(X.shape[0]):
            for j in range(Y.shape[0]):
                K[i, j] = np.exp(-gamma * np.sum(np.square(X[i] - Y[j])))
        return K

    def fit(self, X):
        self.X_fit_ = X
        K = self._rbf_kernel(X, X, self.gamma)
        self._centerer = KernelCenterer().fit(K)
        Kt = self._centerer.transform(K)
        eigval, eigvec = np.linalg.eigh(Kt)
        self.lambdas_ = eigval[-self.n_components:][::-1]
        self.alphas_ = eigvec[:, -self.n_components:][:, ::-1]
        return self

    def transform(self, X):
        K = self._rbf_kernel(X, self.X_fit_, self.gamma)
        Kt = self._centerer.transform(K)
        scaled_alphas = self.alphas_ / np.sqrt(self.lambdas_)
        return np.dot(Kt, scaled_alphas)

In [6]:
X, _ = load_iris(return_X_y=True)
kpca1 = KernelPCA(n_components=2).fit(X)
kpca2 = skKernelPCA(n_components=2, kernel='rbf').fit(X)
assert np.allclose(kpca1.lambdas_, kpca2.lambdas_)
for i in range(kpca1.alphas_.shape[1]):
    assert np.allclose(kpca1.alphas_[:, i], kpca2.alphas_[:, i]) or np.allclose(kpca1.alphas_[:, i], -kpca2.alphas_[:, i])
Xt1 = kpca1.transform(X)
Xt2 = kpca2.transform(X)
for i in range(Xt1.shape[1]):
    assert np.allclose(Xt1[:, i], Xt2[:, i]) or np.allclose(Xt1[:, i], -Xt2[:, i])